In [1]:
import os

os.chdir(path = os.getcwd())

import sys
import numpy as np
import pandas as pd
from math import ceil
from tqdm import trange
from subprocess import call
from itertools import islice
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, dok_matrix

In [2]:
file_dir = 'ml-100k'
file_path = os.path.join(file_dir, 'u.data')
if not os.path.isdir(file_dir):
    call(['curl', '-O', 'http://files.grouplens.org/datasets/movielens/' + file_dir + '.zip'])
    call(['unzip', file_dir + '.zip'])

# we will not be using the timestamp column
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv(file_path, sep = '\t', names = names)
print('data dimension: \n', df.shape)
df.head()

data dimension: 
 (100000, 4)


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
def create_matrix(data, users_col, items_col, ratings_col, threshold = None):
    """
    creates the sparse user-item interaction matrix,
    if the data is not in the format where the interaction only
    contains the positive items (indicated by 1), then use the 
    threshold parameter to determine which items are considered positive
    
    Parameters
    ----------
    data : DataFrame
        implicit rating data

    users_col : str
        user column name

    items_col : str
        item column name
    
    ratings_col : str
        implicit rating column name

    threshold : int, default None
        threshold to determine whether the user-item pair is 
        a positive feedback

    Returns
    -------
    ratings : scipy sparse csr_matrix, shape [n_users, n_items]
        user/item ratings matrix

    data : DataFrame
        implict rating data that retains only the positive feedback
        (if specified to do so)
    """
    if threshold is not None:
        data = data[data[ratings_col] >= threshold]
        data[ratings_col] = 1
    
    for col in (items_col, users_col, ratings_col):
        data[col] = data[col].astype('category')

    ratings = csr_matrix((data[ratings_col],
                          (data[users_col].cat.codes, data[items_col].cat.codes)))
    ratings.eliminate_zeros()
    return ratings, data


In [4]:
def filter_data(data, users_col, items_col, ratings_col, rating_threshold = None, count_threshold = 5):
    """
    choose data with user_id whose item count is
    greater than threshold
    
    Parameters
    ----------
    data : DataFrame
        implicit rating data

    users_col : str
        user column name

    items_col : str
        item column name
    
    threshold : int, default 5
        threshold to determine whether the user is chosen as training data

    Returns
    -------
    data_filter : DataFrame
        Filtered data where a user is eligible to be in training data
    """
    
    data_threshold = None
    if rating_threshold is not None:
        data_threshold = data[data[ratings_col] >= rating_threshold]
        data_threshold[ratings_col] = 1
    
    data_count = data_threshold.groupby(users_col).count()[[items_col]] \
        .reset_index().rename(columns={items_col: 'item_count'})
    
    data_filter = data_threshold \
        .merge(data_count[data_count['item_count'] >= count_threshold][[users_col]], \
               on=users_col)
    
    return data_filter

In [5]:
df_train = filter_data(df, 'user_id', 'item_id', 'rating', 3, 50)

/home/erwin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
X_train, df_train = create_matrix(df_train, 'user_id', 'item_id', 'rating')

In [7]:
X_train

<505x1558 sparse matrix of type '<class 'numpy.int64'>'
	with 70144 stored elements in Compressed Sparse Row format>

In [8]:
df_train

,user_id,item_id,rating,timestamp
0,186,302,1,891717742
1,186,566,1,879023663
2,186,148,1,891719774
3,186,263,1,879023571
4,186,470,1,879023693
...,...,...,...,...
70139,936,312,1,886831853
70140,936,1226,1,886833148
70141,936,251,1,886832134
70142,936,287,1,886832419


In [9]:
df_train.dtypes

user_id      category
item_id      category
rating       category
timestamp       int64
dtype: object

In [10]:
df_train.user_id.cat.codes

0         93
1         93
2         93
3         93
4         93
        ... 
70139    500
70140    500
70141    500
70142    500
70143    500
Length: 70144, dtype: int16

In [11]:
new_df = pd.DataFrame()

In [12]:
new_df['user_id'] = df_train['user_id']

In [13]:
new_df

,user_id
0,186
1,186
2,186
3,186
4,186
...,...
70139,936
70140,936
70141,936
70142,936


In [14]:
new_df['user_index'] = df_train.user_id.cat.codes

In [15]:
new_df = new_df.drop_duplicates().reset_index(drop=True)

In [16]:
new_df

,user_id,user_index
0,186,93
1,298,157
2,253,128
3,305,161
4,6,3
...,...,...
500,938,501
501,940,502
502,942,503
503,943,504


In [17]:
new_df[new_df['user_index'] <= 2]

,user_id,user_index
77,5,2
84,1,0
149,2,1


In [18]:
class BPR:
    """
    Bayesian Personalized Ranking (BPR) for implicit feedback data

    Parameters
    ----------
    learning_rate : float, default 0.01
        learning rate for gradient descent

    n_factors : int, default 20
        Number/dimension of user and item latent factors

    n_iters : int, default 15
        Number of iterations to train the algorithm
        
    batch_size : int, default 1000
        batch size for batch gradient descent, the original paper
        uses stochastic gradient descent (i.e., batch size of 1),
        but this can make the training unstable (very sensitive to
        learning rate)

    reg : int, default 0.01
        Regularization term for the user and item latent factors

    seed : int, default 1234
        Seed for the randomly initialized user, item latent factors

    verbose : bool, default True
        Whether to print progress bar while training

    Attributes
    ----------
    user_factors : 2d ndarray, shape [n_users, n_factors]
        User latent factors learnt

    item_factors : 2d ndarray, shape [n_items, n_factors]
        Item latent factors learnt

    References
    ----------
    S. Rendle, C. Freudenthaler, Z. Gantner, L. Schmidt-Thieme 
    Bayesian Personalized Ranking from Implicit Feedback
    - https://arxiv.org/abs/1205.2618
    """
    def __init__(self, learning_rate = 0.01, n_factors = 15, n_iters = 10, 
                 reg = 0.01, seed = 1234, verbose = True):
        self.reg = reg
        self.seed = seed
        self.verbose = verbose
        self.n_iters = n_iters
        self.n_factors = n_factors
        self.learning_rate = learning_rate
        
        # to avoid re-computation at predict
        self._prediction = None
        
    def fit(self, ratings):
        """
        Parameters
        ----------
        ratings : scipy sparse csr_matrix, shape [n_users, n_items]
            sparse matrix of user-item interactions
        """
        indptr = ratings.indptr
        indices = ratings.indices
        n_users, n_items = ratings.shape
        
        # initialize random weights
        rstate = np.random.RandomState(self.seed)
#         self.user_factors = rstate.normal(size = (n_users, self.n_factors))
#         self.item_factors = rstate.normal(size = (n_items, self.n_factors))
        self.user_factors = (rstate.uniform(size = (n_users, self.n_factors)) - 0.5) / self.n_factors
        self.item_factors = (rstate.uniform(size = (n_items, self.n_factors)) - 0.5) / self.n_factors
        self.item_biases = np.zeros(n_items)
        
        # progress bar for training iteration if verbose is turned on
        loop = range(self.n_iters)
        if self.verbose:
            loop = trange(self.n_iters, desc = self.__class__.__name__)
        
        for _ in loop:
            print('iterate :', _)
            sampled = self._sample(n_users, n_items, indices, indptr)
            sampled_users, sampled_pos_items, sampled_neg_items = sampled
            self._update_2(sampled_users, sampled_pos_items, sampled_neg_items)

        return self
    
    def _sample(self, n_users, n_items, indices, indptr):
        """sample batches of random triplets u, i, j"""
#         sampled_pos_items = np.zeros(n_users, dtype = np.int)
#         sampled_neg_items = np.zeros(n_users, dtype = np.int)
#         sampled_users = np.random.choice(n_users, size = n_users, replace = False)
        
#         for user in sampled_users:
#             pos_items = indices[indptr[user]:indptr[user + 1]]
#             pos_item = np.random.choice(pos_items)
#             neg_item = np.random.choice(n_items)
#             while neg_item in pos_items:
#                 neg_item = np.random.choice(n_items)

#             sampled_pos_items[user] = pos_item
#             sampled_neg_items[user] = neg_item

        random_users = np.random.choice(n_users, size = n_users, replace = False)
        sampled_users = []
        sampled_pos_items = []
        sampled_neg_items = []
        
        for user in random_users:
            pos_items = indices[indptr[user]:indptr[user + 1]]
            pos_item = np.random.choice(pos_items)
            neg_item = np.random.choice(n_items)
            
            if neg_item in pos_items:
                continue
            
            sampled_users.append(user)
            sampled_pos_items.append(pos_item)
            sampled_neg_items.append(neg_item)

        return sampled_users, sampled_pos_items, sampled_neg_items
                
    def _update(self, u, i, j):
        """
        update according to the bootstrapped user u, 
        positive item i and negative item j
        """
        user_u = self.user_factors[u]
        item_i = self.item_factors[i]
        item_j = self.item_factors[j]

        
        bias_i = self.item_biases[i]
        bias_j = self.item_biases[j]
                
        # decompose the estimator, compute the difference between
        # the score of the positive items and negative items; a
        # naive implementation might look like the following:
        # r_ui = np.diag(user_u.dot(item_i.T))
        # r_uj = np.diag(user_u.dot(item_j.T))
        # r_uij = r_ui - r_uj
        
        # however, we can do better, so
        # for batch dot product, instead of doing the dot product
        # then only extract the diagonal element (which is the value
        # of that current batch), we perform a hadamard product, 
        # i.e. matrix element-wise product then do a sum along the column will
        # be more efficient since it's less operations
        # http://people.revoledu.com/kardi/tutorial/LinearAlgebra/HadamardProduct.html
        # r_ui = np.sum(user_u * item_i, axis = 1)
        #
        # then we can achieve another speedup by doing the difference
        # on the positive and negative item up front instead of computing
        # r_ui and r_uj separately, these two idea will speed up the operations
        # from 1:14 down to 0.36

        r_uij = np.sum(user_u * (item_i - item_j), axis = 1)

        r_uij = r_uij + (bias_i - bias_j)
        
        sigmoid = 1.0 / (1.0 + np.exp(r_uij))
        
        print('correct : {}% with {} users'.format(np.sum(sigmoid < 0.5) * 100.0 / sigmoid.shape[0], sigmoid.shape[0]))
        print('skipped : {}% with {} users'.format((505 - sigmoid.shape[0]) * 100.0 / 505, 505 - sigmoid.shape[0]))
        
        # repeat the 1 dimension sigmoid n_factors times so
        # the dimension will match when doing the update
        sigmoid_tiled = np.tile(sigmoid, (self.n_factors, 1)).T

        # update using gradient descent
        grad_user_u = sigmoid_tiled * (item_i - item_j) - self.reg * user_u
        grad_item_i = sigmoid_tiled * user_u            - self.reg * item_i
        grad_item_j = sigmoid_tiled * -user_u           - self.reg * item_j
        grad_bias_i = sigmoid - self.reg * bias_i
        grad_bias_j = -sigmoid - self.reg * bias_j
        
        # update users and items factors
        self.user_factors[u] += self.learning_rate * grad_user_u
        self.item_factors[i] += self.learning_rate * grad_item_i
        self.item_factors[j] += self.learning_rate * grad_item_j

        # update item biases
        self.item_biases[i] += self.learning_rate * grad_bias_i
        self.item_biases[j] += self.learning_rate * grad_bias_j
            
        return self
    
    def _update_2(self, u, i, j):
        assert len(u) == len(i)
        assert len(u) == len(j)
        
        correct = 0
        n = len(u)
        for idx in range(n):
            u_id = u[idx]
            i_id = i[idx]
            j_id = j[idx]
            
            user_u = self.user_factors[u_id]
            item_i = self.item_factors[i_id]
            item_j = self.item_factors[j_id]
            
            bias_i = self.item_biases[i_id]
            bias_j = self.item_biases[j_id]
            
            score = bias_i - bias_j
            score += np.dot(user_u, (item_i - item_j).T)
            
            z = 1.0 / (1.0 + np.exp(score))
            
            if z < 0.5:
                correct += 1
            
            grad_user_u = z * (item_i - item_j) - self.reg * user_u
            grad_item_i = z * user_u - self.reg * item_i
            grad_item_j = -z * user_u - self.reg * item_j
            
            grad_bias_i = z - self.reg * bias_i
            grad_bias_j = -z - self.reg * bias_j

            
            self.user_factors[u_id] += self.learning_rate * grad_user_u
            self.item_factors[i_id] += self.learning_rate * grad_item_i
            self.item_factors[j_id] += self.learning_rate * grad_item_j
            
            self.item_biases[i_id] += self.learning_rate * grad_bias_i
            self.item_biases[j_id] += self.learning_rate * grad_bias_j
        
        print('correct :', correct * 100.0 / len(u))
            
        return self


In [19]:
class MYBPR:
    def __init__(self, learning_rate = 0.01, n_factors = 15, n_iters = 10, 
                 reg = 0.01, seed = 1234, verbose = True):
        self.reg = reg
        self.seed = seed
        self.verbose = verbose
        self.n_iters = n_iters
        self.n_factors = n_factors
        self.learning_rate = learning_rate
        
        # to avoid re-computation at predict
        self._prediction = None
    
    def _prepare_data(self, ratings):
        user_counts = np.ediff1d(ratings.indptr)
        user_ids = np.repeat(np.arange(ratings.shape[0]), user_counts).astype(ratings.indices.dtype)
        
        return user_counts, user_ids
    
        
    def fit(self, ratings):
        indptr = ratings.indptr
        indices = ratings.indices
        n_users, n_items = ratings.shape
        
        user_counts, user_ids = self._prepare_data(ratings)
        neg_item_ids = np.arange(n_items, dtype=np.int32)
        
        # initialize random weights
        rstate = np.random.RandomState(self.seed)
        self.user_factors = (rstate.uniform(size = (n_users, self.n_factors)) - 0.5) / self.n_factors
        self.item_factors = (rstate.uniform(size = (n_items, self.n_factors)) - 0.5) / self.n_factors
        self.item_biases = np.zeros(n_items)
        
        # progress bar for training iteration if verbose is turned on
        loop = range(self.n_iters)
        if self.verbose:
            loop = trange(self.n_iters, desc = self.__class__.__name__)
        
        for _ in loop:
            print('iterate :', _)
            correct, skipped = self._fit_sgd(user_ids, indices, neg_item_ids, indptr)
            print(' correct : {:.2f}%'.format((100.0 * correct / (len(user_ids) - skipped))))
            print(' skipped : {:.2f}%'.format((100.0 * skipped / len(user_ids))))

        return self

    
    def _fit_sgd(self, user_ids, item_ids, neg_item_ids, indptr):
        correct = 0
        skipped = 0
        num_samples = len(user_ids)
        
        for _ in range(num_samples):
            i_index = np.random.choice(num_samples)
            i_id = item_ids[i_index]
            
            j_id = np.random.choice(neg_item_ids)
            
            u_id = user_ids[i_index]
            
            pos_items = item_ids[indptr[u_id]:indptr[u_id + 1]]
            if j_id in pos_items:
                skipped += 1
                continue
            
            user_u = self.user_factors[u_id]
            item_i = self.item_factors[i_id]
            item_j = self.item_factors[j_id]
            
            bias_i = self.item_biases[i_id]
            bias_j = self.item_biases[j_id]
            
            score = bias_i - bias_j
            score += np.dot(user_u, (item_i - item_j).T)
            
            z = 1.0 / (1.0 + np.exp(score))
            
            if z < 0.5:
                correct += 1
            
            grad_user_u = z * (item_i - item_j) - self.reg * user_u
            grad_item_i = z * user_u - self.reg * item_i
            grad_item_j = -z * user_u - self.reg * item_j
            
            grad_bias_i = z - self.reg * bias_i
            grad_bias_j = -z - self.reg * bias_j

            
            self.user_factors[u_id] += self.learning_rate * grad_user_u
            self.item_factors[i_id] += self.learning_rate * grad_item_i
            self.item_factors[j_id] += self.learning_rate * grad_item_j
            
            self.item_biases[i_id] += self.learning_rate * grad_bias_i
            self.item_biases[j_id] += self.learning_rate * grad_bias_j 
        
        return correct, skipped

In [20]:
# parameters were randomly chosen
bpr_params = {
    'reg': 0.001,
    'learning_rate': 0.01,
    'n_iters': 10,
    'n_factors': 30,
    'seed': 42
}

bpr = BPR(**bpr_params)
bpr.fit(X_train)

BPR:  40%|████      | 4/10 [00:00<00:00, 37.33it/s]

iterate : 0
correct : 62.723214285714285
iterate : 1
correct : 71.93763919821826
iterate : 2
correct : 74.78260869565217
iterate : 3
correct : 74.89451476793249
iterate : 4
correct : 79.9554565701559
iterate : 5
correct : 80.56155507559396
iterate : 6
correct : 79.12087912087912
iterate : 7
correct : 77.15517241379311
iterate : 8
correct :

BPR: 100%|██████████| 10/10 [00:00<00:00, 40.43it/s]

 82.31441048034935
iterate : 9
correct : 75.32467532467533


In [21]:
# parameters were randomly chosen
bpr_params = {
    'reg': 0.001,
    'learning_rate': 0.01,
    'n_iters': 50,
    'n_factors': 30,
    'seed': 42
}

bpr = MYBPR(**bpr_params)
bpr.fit(X_train)

MYBPR:   0%|          | 0/50 [00:00<?, ?it/s]

iterate : 0


MYBPR:   2%|▏         | 1/50 [00:03<02:38,  3.24s/it]

 correct : 83.53%
 skipped : 11.78%
iterate : 1


MYBPR:   4%|▍         | 2/50 [00:06<02:34,  3.22s/it]

...

 correct : 89.81%
 skipped : 11.91%
iterate : 48


MYBPR:  98%|█████████▊| 49/50 [02:46<00:03,  3.63s/it]

 correct : 89.83%
 skipped : 11.95%
iterate : 49


MYBPR: 100%|██████████| 50/50 [02:49<00:00,  3.38s/it]

 correct : 90.00%
 skipped : 12.06%


# Compare to cornac

In [22]:
import cornac

In [23]:
df_train

,user_id,item_id,rating,timestamp
0,186,302,1,891717742
1,186,566,1,879023663
2,186,148,1,891719774
3,186,263,1,879023571
4,186,470,1,879023693
...,...,...,...,...
70139,936,312,1,886831853
70140,936,1226,1,886833148
70141,936,251,1,886832134
70142,936,287,1,886832419


In [24]:
train_set = cornac.data.Dataset.from_uir(df_train[['user_id', 'item_id', 'rating']].itertuples(index=False), seed=42)

In [25]:
train_set

In [26]:
cornac_bpr = cornac.models.BPR(k=30,
                        max_iter=50,
                        learning_rate=0.01,
                        lambda_reg=0.001,
                        verbose=True,
                        seed=42)

In [27]:
cornac_bpr.fit(train_set)


Optimization finished!
